**PyTorch has two primitives to work with data: `torch.utils.data.DataLoader` and `torch.utils.data.Dataset` <br> Dataset stores the samples and their corresponding labels, and DataLoader wraps an iterable around the Dataset**

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [2]:
#Download training data from open datasets
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

#Download the test data from open datasets
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

- **We pass the dataset as an argument to dataloader**
- **This wraps an iterable over our dataset, and supports automatic batching, sampling, shuffling and multiprocess data loading**
- **Here we define a batch size of 64, i.e. each element in the dataloader iterable will return a batch of 64 features and labels**

In [3]:
batch_size = 64

#Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("shape of y: ", y.shape, y.dtype)
    break


Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
shape of y:  torch.Size([64]) torch.int64


- **To define a neural network in PyTorch, we create a class that inherits from `nn.Module`.**
- **We define the layers of the network in the `__init__` function and specify how data will pass through the network in the `forward` function.**
- **To accelerate operations in the neural network, we move it tothte GPU if available.**

In [4]:
#Get cpu or gpu device for training
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

#Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


**To train a model, we need a loss function and an optimizer.**

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

**In a single training loop, the model makes predictions on the training dataset (fet to it in batches), and <br>
backpropagates the prediction error to adjust the model's parameters.**

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        #compute the prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        
        #Backpropagtion
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
            

**We also check the model's performance against the test dataset to ensure it is learning**

In [7]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= size
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n -------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done")

Epoch 1
 -------------------------------
loss: 2.310527 [    0/60000]
loss: 2.301631 [ 6400/60000]
loss: 2.295037 [12800/60000]
loss: 2.281919 [19200/60000]
loss: 2.285928 [25600/60000]
loss: 2.278539 [32000/60000]
loss: 2.268762 [38400/60000]
loss: 2.266127 [44800/60000]
loss: 2.257108 [51200/60000]
loss: 2.223056 [57600/60000]
Test Error: 
 Accuracy: 36.4%, Avg loss: 0.035175 

Epoch 2
 -------------------------------
loss: 2.270846 [    0/60000]
loss: 2.269924 [ 6400/60000]
loss: 2.242002 [12800/60000]
loss: 2.208688 [19200/60000]
loss: 2.231226 [25600/60000]
loss: 2.211872 [32000/60000]
loss: 2.196250 [38400/60000]
loss: 2.196488 [44800/60000]
loss: 2.185584 [51200/60000]
loss: 2.112997 [57600/60000]
Test Error: 
 Accuracy: 39.9%, Avg loss: 0.033837 

Epoch 3
 -------------------------------
loss: 2.223381 [    0/60000]
loss: 2.225564 [ 6400/60000]
loss: 2.171210 [12800/60000]
loss: 2.097565 [19200/60000]
loss: 2.148627 [25600/60000]
loss: 2.125655 [32000/60000]
loss: 2.085467 [384

**A common way to save a model is to serialize the internaal state dictionary (containing the model paaramters)**

In [9]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


**The process for loading a model includes re-creating the model structure and loading the state dictionary into it**

In [10]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

**This model can now be used to make predictions**

In [11]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Sneaker", Actual: "Ankle boot"
